# Лабораторна робота 2 з Теоретико-числових алгоритмів в криптології
## Тема: Реалiзацiя та застосування алгоритму дискретного логарифмування **Silver–Pohlig–Hellman**

**Виконали:** Бондар Петро, Кістаєв Матвій\
**Група:** ФІ-03

### Посилання
Github: https://github.com/Pechenkya/NTA-Labs-Bondar-Kistaiev-FI-03/tree/main/Lab-2


In [76]:
import math

from factor_module import check_prime

## Допоміжні математичні функції

#### Алгоритм розв'язання системи порівнянь по модулю за Китайською Теоремою про Лишки

In [77]:
# Solves system of congruences x = a[i] mod n[i] by Chinese Remainder Theorem

def CRT(a, n):
    n_prod = math.prod(n)
    N = [n_prod // n_i for n_i in n]
    M = [pow(n_prod // n_i, -1, n_i) for n_i in n]

    return sum([a[i]*M[i]*N[i] for i in range(0, len(a))]) % n_prod


#### Функції, що генерують усі прості числа до заданого N та розкладу числа на прості множники

In [78]:
# Generates all primes from 2 to n
def PrimesRange(n):
    P = [2]
    for a in range(3, n):
        if check_prime(a):
            P.append(a)
    
    return P

def NextPrime(n):
    p = n + 1
    while not check_prime(p):
        p += 1

    return p

# Factors n 
def Factor(n):
    F = {}
    p_i = 2

    while n != 1:
        k = 0
        while n % (p_i ** (k+1)) == 0:
            k += 1

        if k > 0:
            F[p_i] = k
            n = n // (p_i**k)
        
        p_i = NextPrime(p_i)

    if n != 1:
        F[n] = 1

    return F

## Алгоритм Сільвера-Поліга-Гелмана

In [79]:
# Computes discrete log_a(b) modp by Silver–Pohlig–Hellman algorithm

def SPH(a, b, p):
    n = p - 1
    F = Factor(n)
    r_table = {}

    for p_i in F.keys():
        for j in range(0, p_i):
            t = pow(a, (n * j) // p_i, p)
            r_table[p_i, t] = j
    
    Y = []
    P = [p_i**l_i for p_i, l_i in F.items()]

    for p_i in F.keys():
        a_x_modn = 1
        x = 0
        for j in range(0, F[p_i]):
            t = pow(b * pow(a_x_modn, -1, p), n // (p_i ** (j+1)), p)
            x_j = r_table[p_i, t]
            x += (x_j * (p_i ** j)) % (p_i ** F[p_i])
            a_x_modn = (a_x_modn * pow(a, x_j * (p_i ** j), p)) % p
        
        Y.append(x)
    
    return CRT(Y, P)

a = 40339922
p = 46835927
b = 8665466
print(Factor(p-1))
x = SPH(a, b, p)
print(x)

print(pow(a, x, p) == b)

{2: 1, 157: 1, 149159: 1}
36750957
True


## Порівняння із повним перебором

#### Алгоритм, що розв'язує задачу дискретного логарифмування повним перебором

In [80]:
def BruteDL(a, b, p):
    ax = 1
    x = 0
    for i in range(0, p):
        if ax == b: return x

        ax = (ax * a) % p
        x += 1

    return float('nan')

print(BruteDL(a, b, p))

36750957


### Заміри часу виконання і порівняння з рішенням з другої лабораторної роботи

$$ \begin{array}{|c|c|r|r|r|r|c|c|c|}
    \hline № & \text{Test Level} & \alpha & \beta & \mod & \text{Solution} & \text{SPH time (sec)} & \text{Index-Calculus time (sec)} & \text{Bruteforce (sec)}\\
    \hline 1 & 3\;(1) & 405 & 162 & 863 & 602 & 0.0046                      & 0.0040 & 0.0000\\
    \hline 2 & 3\;(2) & 91 & 54 & 983 & 490 &  0.0045                        & 0.0036& 0.0000\\
    \hline 3 & 4\;(1) & 1735 & 3267 & 5443 & 2877   & 0.0090                 & 0.0062& 0.0000\\
    \hline 4 & 4\;(2) & 5606 & 766 & 5807 & 5787 & 0.0338                    & 0.0050& 0.0000\\
    \hline 5 & 5\;(1) & 53100 & 16911 & 64567 & 47387   & 0.0020             & 0.0159& 0.0030\\
    \hline 6 & 5\;(2) & 236 & 21489 & 32069 & 25264 & 0.0730                 & 0.0095& 0.0020\\
    \hline 7 & 6\;(1) & 324217 & 96569 & 397519 & 292529 & 0.0025            & 0.0251& 0.0217\\
    \hline 8 & 6\;(2) & 341681 & 719645 & 933397 & 360413 & 0.7903           & 0.0306& 0.0266\\
    \hline 9 & 7\;(1) & 416859 & 811893 & 1100641 & 849195 & 0.0216          & 0.0491& 0.0600\\
    \hline 10 & 7\;(2) & 698063 & 1842530 & 4948849 & 4639227 & 0.3340       & 0.0502& 0.3651\\
    \hline 11 & 8\;(1) & 10329729 & 5996667 & 12879569 & 3681443 & 0.0319    & 0.0643& 0.2640\\
    \hline 12 & 8\;(2) & 40339922 & 8665466 & 46835927 & 36750957 & 1.6040   & 0.1106& 2.6295\\
    \hline 13 & 9\;(1) & 133831429 & 394869515 & 603047449 & 575130035 & 0.0110 & 0.3050& 42.387\\
    \hline 14 & 9\;(2) & 105885086 & 242684147 & 281243987 & 116787747 & 0.1408 & 0.2262& 8.6493\\
    \hline 15 & 10\;(1) & 3254326800 & 5738480278 & 6821486569 & 5425105225 & 0.1868                        & 0.8527& -\\
    \hline 16 & 10\;(2) & 2503970251 & 4616555134 & 7687535143 & 3612269744 & 0.7432                        & 0.8898&-\\
    \hline 17 & 11\;(1) & 15755281373 & 2216307038 & 27834916511 & 3740804930 & 0.0680                      & 1.3528&-\\
    \hline 18 & 11\;(2) & 22853910777 & 25817503193 & 28791230497 & 21096709669 &  -                         & 1.4848&-\\
    \hline 19 & 12\;(1) & 507709796487 & 765316699585 & 808811929619 & 454007131294 & 0.0195                     & 5.8950&-\\
    \hline 20 & 12\;(2) & 103462240942 & 347520779949 & 423107943311 & 104026002582 & 0.8080                     & 4.9429&-\\
    \hline 21 & 13\;(1) & 575769640533 & 1587103349320 & 2679503891797 & 2508924257003 & 0.2689                  & 11.157&-\\
    \hline 22 & 13\;(2) & 5193719065806 & 433642518389 & 5200738710043 & 3299582449828 & -                  & 12.986&-\\
    \hline 23 & 14\;(1) & 69729143465395 & 51717474377131 & 81827408544181 & 70597713698775 & 0.1735             & 36.710&-\\
    \hline 24 & 14\;(2) & 9342677045103 & 5678740808699 & 34191150382817 & 26463769397737 & -               & 26.645&-\\
    \hline 25 & 15\;(1) & 58035538961226 & 84400445924237 & 124360974085129 & 13651448394237 & 0.2443            & 48.763&-\\
    \hline 26 & 15\;(2) & 137042521277099 & 481703362714446 & 648235951112423 & 266893171218140 & -         & 85.634&-\\
    \hline 27 & 16\;(1) & 1646807569044576 & 1357634646373857 & 2292878607986357 & 547317542238434 & 0.0864      & 152.45&-\\
    \hline 28 & 16\;(2) & 783529233531527 & 1339896166988741 & 1486639524290603 & 318864505892145 & -       & 129.01&-\\
    \hline
\end{array}$$

Числа довжини 17 і більше дуууже довго генеруються допоміжною програмою :)

#### Графіки (Кількість цифр - Час роботи)

![pic1](../Images/small.png "Title")

![pic1](../Images/big.png "Title")